In [3]:
import pandas as pd  


In [4]:
outgoing_df = pd.read_csv("../Data/Köln/importantStructureOutgoing.csv")

In [5]:
outgoing_df.groupby("base").count()

,link,type
base,,
50 Cent,29,29
Aaron Lebedeff,29,29
Act Up,28,28
Al Harewood,30,30
Alexander Jackson Davis,26,26
...,...,...
Werner Janssen (Komponist),29,29
Westminster Kennel Club Dog Show,28,28
William Beach Lawrence,29,29


In [107]:
views_df = pd.read_csv("../Data/Köln/views.csv")
views_df[(views_df["views"] > 100) & (views_df["link"].isin(outgoing_df["base"]))]

,link,date,views
49656,Krieg von Castellammare,2023092500,132
53038,Bonnie Bedelia,2023070600,112
53055,Bonnie Bedelia,2023072300,192
53058,Bonnie Bedelia,2023072600,185
53062,Bonnie Bedelia,2023073000,162
...,...,...,...
180048,John Barrett (Friseur),2023082100,214
180049,John Barrett (Friseur),2023082200,104
182507,Internationales Hot-Dog-Wettessen,2023070400,271
182508,Internationales Hot-Dog-Wettessen,2023070500,339


Fill all empty dates in view_df with 0

In [7]:
all_dates = views_df[views_df["link"] == "50 Cent"]["date"]
# for link in views_df["link"]:
#   merged = 
#filled_views_df = list(map(lambda link: pd.merge(views_df[views_df["link"] == link], all_dates, how="right", on="date").fillna({"link": link, "views": 0}), list(views_df["link"])))

In [8]:
all_dates = views_df[views_df["link"] == "50 Cent"]["date"]
filled_list = []
for link in views_df["link"].unique():
  filled_list.append(views_df[views_df["link"] == link].merge(all_dates,how="right", on="date").fillna({"link": link, "views": 0}))

filled_df = pd.concat(filled_list, ignore_index=True)

filled_df.to_csv("../Data/Köln/filled_views.csv", index=False)
  

In [9]:
filled_df

,link,date,views
0,FIFA-Weltfußballer des Jahres,2023070100,270.0
1,FIFA-Weltfußballer des Jahres,2023070200,871.0
2,FIFA-Weltfußballer des Jahres,2023070300,232.0
3,FIFA-Weltfußballer des Jahres,2023070400,215.0
4,FIFA-Weltfußballer des Jahres,2023070500,198.0
...,...,...,...
203572,COVID-19-Pandemie,2023092700,913.0
203573,COVID-19-Pandemie,2023092800,876.0
203574,COVID-19-Pandemie,2023092900,811.0
203575,COVID-19-Pandemie,2023093000,715.0


In [10]:
links = pd.read_csv("../Data/Köln/importantStructureOutgoing.csv")
links

,base,link,type
0,Charlie Barnett (Komiker),1954,outgoing
1,Charlie Barnett (Komiker),Komiker,outgoing
2,Charlie Barnett (Komiker),Afroamerikaner,outgoing
3,Charlie Barnett (Komiker),New York City,outgoing
4,Charlie Barnett (Komiker),West Virginia,outgoing
...,...,...,...
2545,Michael Blake (Musiker),Lounge Lizards,outgoing
2546,Michael Blake (Musiker),Billy Martin (Schlagzeuger),outgoing
2547,Michael Blake (Musiker),David Tronzo,outgoing
2548,Michael Blake (Musiker),Klarinette,outgoing


In [108]:
combination = filled_df.merge(links,on="link",how="right")
ml_output = []
ml_link = []
for base in links["base"].sort_values().unique():
  ml_output += filled_df[filled_df["link"] == base].sort_values("date")["views"].to_list()
  ml_link += filled_df[filled_df["link"] == base]["link"].to_list()
len(ml_link)

9300

In [112]:
res = []
for idx in range(0, len(ml_output)) :
    if ml_output[idx] > 100:
        res.append(idx)

In [13]:
combination = combination.sort_values(["base", "date"], ignore_index=True)
ml_input = []
for base in combination["base"].unique():
  for date in combination["date"].unique():
    ml_input.append(combination[(combination["base"] == base) & (combination["date"] == date)])


In [163]:
ml_input[2]["date"]

timestamp = 2023070100
datetime = pd.to_datetime(timestamp, format='%Y%m%d%H')

print(datetime)

58    2023070300
59    2023070300
60    2023070300
61    2023070300
62    2023070300
63    2023070300
64    2023070300
65    2023070300
66    2023070300
67    2023070300
68    2023070300
69    2023070300
70    2023070300
71    2023070300
72    2023070300
73    2023070300
74    2023070300
75    2023070300
76    2023070300
77    2023070300
78    2023070300
79    2023070300
80    2023070300
81    2023070300
82    2023070300
83    2023070300
84    2023070300
85    2023070300
86    2023070300
Name: date, dtype: int64

In [164]:
ml_input_value = [e["views"].tolist() for e in ml_input]

TypeError: '_AtIndexer' object is not callable

In [78]:
ml_input_value[0]

[1556.0,
 174.0,
 22.0,
 44.0,
 108.0,
 221.0,
 4378.0,
 228.0,
 186.0,
 37.0,
 8.0,
 1003.0,
 522.0,
 101.0,
 22.0,
 133.0,
 40.0,
 57.0,
 36.0,
 0.0,
 15.0,
 61.0,
 1718.0,
 89.0,
 53.0,
 989.0,
 26.0,
 238.0,
 84.0]

In [80]:
import numpy as np

for i in range(len(ml_input_value)):
  if len(ml_input_value[i]) < 30:
    zero_list = list(np.empty(30- len(ml_input_value[i])))
    ml_input_value[i] = ml_input_value[i] + zero_list

ml_input_value

[[1556.0,
  174.0,
  22.0,
  44.0,
  108.0,
  221.0,
  4378.0,
  228.0,
  186.0,
  37.0,
  8.0,
  1003.0,
  522.0,
  101.0,
  22.0,
  133.0,
  40.0,
  57.0,
  36.0,
  0.0,
  15.0,
  61.0,
  1718.0,
  89.0,
  53.0,
  989.0,
  26.0,
  238.0,
  84.0,
  116.0],
 [1981.0,
  159.0,
  28.0,
  70.0,
  115.0,
  179.0,
  4792.0,
  200.0,
  226.0,
  31.0,
  4.0,
  1033.0,
  468.0,
  137.0,
  18.0,
  90.0,
  25.0,
  48.0,
  42.0,
  0.0,
  17.0,
  102.0,
  1589.0,
  86.0,
  46.0,
  1026.0,
  40.0,
  326.0,
  57.0,
  116.0],
 [1800.0,
  96.0,
  19.0,
  67.0,
  79.0,
  174.0,
  4872.0,
  161.0,
  209.0,
  14.0,
  3.0,
  736.0,
  422.0,
  111.0,
  26.0,
  73.0,
  19.0,
  63.0,
  20.0,
  0.0,
  8.0,
  160.0,
  1440.0,
  58.0,
  31.0,
  985.0,
  50.0,
  321.0,
  57.0,
  116.0],
 [1715.0,
  119.0,
  20.0,
  76.0,
  88.0,
  182.0,
  5710.0,
  160.0,
  191.0,
  27.0,
  2.0,
  673.0,
  454.0,
  115.0,
  36.0,
  78.0,
  19.0,
  44.0,
  17.0,
  0.0,
  14.0,
  354.0,
  1424.0,
  53.0,
  40.0,
  1395.0,
  27.0,

In [114]:
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(ml_input_value[0:4000],ml_output[0:4000])

DecisionTreeClassifier()

In [118]:
#clf.predict(ml_input_value[7900:8000])
ml_input_value[7952]

[102.0,
 152.0,
 0.0,
 2.0,
 2.0,
 92.0,
 5.0,
 4849.0,
 103.0,
 23.0,
 36.0,
 54.0,
 86.0,
 17.0,
 27.0,
 353.0,
 164.0,
 74.0,
 123.0,
 23.0,
 124.0,
 86.0,
 125.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [152]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


scaler = StandardScaler()

X = scaler.fit_transform(ml_input_value)
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = DecisionTreeRegressor(random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [ 13.   0.   0. ... 218.   0.  28.]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.0, 4.0, 1.0, 10.0, 0.0, 0.0, 27.0, 1

In [150]:
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics


scaler = StandardScaler()

X = scaler.fit_transform(ml_input_value)
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = DecisionTreeRegressor(random_state=0)

model.fit(X_train, y_train)
sel = SelectFromModel(model)
X_trans = sel.fit_transform(X_train, y_train)

model.fit(X_trans, y_train)

X_test_trans = sel.transform(X_test)


# make predictions
y_pred = model.predict(X_test_trans)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [  7.   0.   2. ... 207.   1.  29.]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.0, 4.0, 1.0, 10.0, 0.0, 0.0, 27.0, 1

In [147]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


scaler = StandardScaler()

X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestRegressor(n_estimators=100, random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [1.0950e+01 9.0000e-02 1.0600e+00 ... 1.9288e+02 1.0300e+00 4.6660e+01]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.

In [153]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


scaler = StandardScaler()

X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestRegressor(n_estimators=200, random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [ 11.01    0.235   1.055 ... 197.165   1.025  45.355]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.0, 4.0, 1.0, 10.0,

In [157]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics


scaler = StandardScaler()

X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestRegressor(n_estimators=100, random_state=0)

model.fit(X_train, y_train)
sel = SelectFromModel(model)
X_trans = sel.fit_transform(X_train, y_train)

model.fit(X_trans, y_train)

X_test_trans = sel.transform(X_test)

# make predictions
y_pred = model.predict(X_test_trans)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

Predicted: [ 11.41   0.2    0.93 ... 198.33   1.96  40.9 ]
Actual: [15.0, 0.0, 2.0, 1.0, 1.0, 0.0, 561.0, 0.0, 103.0, 49.0, 2.0, 2.0, 0.0, 0.0, 0.0, 1.0, 606.0, 120.0, 0.0, 0.0, 524.0, 0.0, 28.0, 4.0, 164.0, 6.0, 0.0, 9.0, 0.0, 1.0, 22.0, 0.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 21.0, 1.0, 0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 2.0, 2.0, 2.0, 1.0, 4.0, 1.0, 0.0, 3.0, 0.0, 1.0, 33.0, 0.0, 12.0, 0.0, 3.0, 3.0, 7.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 8.0, 1.0, 16.0, 1.0, 1.0, 0.0, 2.0, 0.0, 3.0, 2.0, 27.0, 204.0, 0.0, 1.0, 0.0, 0.0, 0.0, 4.0, 2.0, 3.0, 0.0, 0.0, 0.0, 0.0, 0.0, 29.0, 1.0, 0.0, 2.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 10.0, 1.0, 35.0, 2.0, 1.0, 18.0, 3.0, 2.0, 0.0, 0.0, 0.0, 2.0, 1.0, 1.0, 4.0, 6.0, 0.0, 64.0, 6.0, 6.0, 0.0, 0.0, 1.0, 119.0, 0.0, 13.0, 25.0, 0.0, 0.0, 0.0, 30.0, 6.0, 0.0, 24.0, 139.0, 3.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 1.0, 7.0, 0.0, 0.0, 9.0, 0.0, 1.0, 0.0, 0.0, 0.0, 2.0, 0.0, 0.0, 2.0, 0.0, 0.0, 5.0, 0.0, 3.0, 4.0, 2.0, 3.0, 162.0, 38.0, 0.0, 0.0, 0.0, 4.0, 1.0, 10.0, 0.0, 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn import metrics


X = ml_input_value
y = ml_output

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

model = RandomForestRegressor(n_estimators=200, random_state=0)

model.fit(X_train, y_train)

# make predictions
y_pred = model.predict(X_test)

# print the results
print('Predicted:', y_pred)
print('Actual:', y_test)

print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))